### Datasetda mavjud bo'lmagan kunlarni qo'shish

In [1]:
print('hi')

hi


In [3]:
import pandas as pd
from pathlib import Path

In [2]:

pd.set_option('display.float_format', lambda v: f'{v:,.2f}')

In [4]:
data_path = Path('datasets/usd_rates60.csv')
df = pd.read_csv(data_path)
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
df = df.sort_values(['Ccy', 'Date']).dropna(subset=['Date']).reset_index(drop=True)
print(f'Satrlar: {len(df):,}; Ustunlar: {df.shape[1]}')
df.head()


Satrlar: 58; Ustunlar: 11


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
0,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,079.28",40.28,2018-09-25
1,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,079.28",40.28,2018-09-25
2,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,179.66",100.38,2018-10-03
3,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,179.66",100.38,2018-10-03
4,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,179.66",100.38,2018-10-03


In [5]:
gap_summary = []
for ccy, g in df.groupby('Ccy'):
    g = g.sort_values('Date')
    gaps = (g['Date'].diff().dt.days.fillna(0) - 1).clip(lower=0)
    gap_summary.append({'Ccy': ccy, 'missing_days': int(gaps.sum())})

gap_df = pd.DataFrame(gap_summary).sort_values('missing_days', ascending=False)
gap_df


,Ccy,missing_days
0,USD,54


In [6]:
def fill_missing_days(group: pd.DataFrame) -> pd.DataFrame:
    group = group.sort_values('Date').drop_duplicates(subset='Date', keep='last')
    full_idx = pd.date_range(group['Date'].min(), group['Date'].max(), freq='D')
    filled = group.set_index('Date').reindex(full_idx).ffill()
    filled['Date'] = filled.index
    return filled.reset_index(drop=True)

filled_df = df.groupby('Ccy', group_keys=False).apply(fill_missing_days)
filled_df = filled_df.sort_values(['Ccy', 'Date']).reset_index(drop=True)

print(f'Oldingi satrlar: {len(df):,}; Yangi satrlar: {len(filled_df):,}')
filled_df.tail(10)


Oldingi satrlar: 58; Yangi satrlar: 64


C:\Users\zikru\AppData\Local\Temp\ipykernel_22912\1201277546.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_df = df.groupby('Ccy', group_keys=False).apply(fill_missing_days)


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
54,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,271.27",21.30,2018-11-18
55,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,271.27",21.30,2018-11-19
56,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-20
57,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-21
58,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-22
59,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-23
60,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-24
61,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-25
62,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-26
63,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,299.20",21.67,2018-11-27


In [7]:
has_gap = (
    filled_df.sort_values(['Ccy', 'Date'])
    .groupby('Ccy')['Date']
    .diff()
    .dt.days
    .fillna(1)
    .gt(1)
    .any()
)
print('Bo`shliq qoldimi?:', bool(has_gap))
filled_df.tail()


Bo`shliq qoldimi?: False


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
59,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-23
60,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-24
61,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-25
62,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,277.53",6.26,2018-11-26
63,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"8,299.20",21.67,2018-11-27


### Lag, rolling va calendar featurelar 

Quyidagi kod `rate` bo'yicha 1, 7 va 30 kunlik lag va rolling xususiyatlarni hosil qiladi.

In [6]:
# Faylni yuklaymiz
data_path = Path('datasets/usd_rates_filled.csv')
df = pd.read_csv(data_path)

# Ustun nomlarini qulay foydalanish uchun kichik harflarga o'girib olamiz
df.columns = [c.lower() for c in df.columns]

# Sanani datetime formatiga o'tkazib, vaqt bo'yicha tartiblaymiz
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Faqat kerakli ustunlarni olamiz: sana va kurs (rate)
df = df[['rate', 'date', 'diff']].reset_index(drop=True)

df.head()

,rate,date,diff
0,8226.10,2018-10-26,18.06
1,8226.10,2018-10-27,18.06
2,8226.10,2018-10-28,18.06
3,8226.10,2018-10-29,18.06
4,8236.87,2018-10-30,10.77


In [8]:
# Lag va rolling oynalar (kunlarda)
lags = [1, 7, 30]
windows = [7, 30]

features = df.copy()

# ----------------------------
# 2) Lag featurelar
# ----------------------------
for lag in lags:
    features[f'lag_{lag}'] = features['rate'].shift(lag)


# ----------------------------
# 3) Rolling window featurelar (100% leakage-free)
# ----------------------------
rate_shifted = features['rate'].shift(1)   # bugungi qiymatni bilmaslik uchun

for win in windows:
    features[f'roll_min_{win}']    = rate_shifted.rolling(win).min()
    features[f'roll_max_{win}']    = rate_shifted.rolling(win).max()
    features[f'roll_mean_{win}']   = rate_shifted.rolling(win).mean()
    features[f'roll_median_{win}'] = rate_shifted.rolling(win).median()
    features[f'roll_std_{win}']    = rate_shifted.rolling(win).std()

features_clean = features.dropna().reset_index(drop=True)
features_clean.head(10)


,rate,date,diff,lag_1,lag_7,lag_30,roll_min_7,roll_max_7,roll_mean_7,roll_median_7,roll_std_7,roll_min_30,roll_max_30,roll_mean_30,roll_median_30,roll_std_30
0,8277.53,2018-11-25,6.26,8277.53,8271.27,8226.10,8271.27,8277.53,8275.741429,8277.53,3.054567,8226.10,8277.53,8253.294000,8249.97,18.682932
1,8277.53,2018-11-26,6.26,8277.53,8271.27,8226.10,8271.27,8277.53,8276.635714,8277.53,2.366058,8226.10,8277.53,8255.008333,8249.97,18.459843
2,8299.20,2018-11-27,21.67,8277.53,8277.53,8226.10,8277.53,8277.53,8277.530000,8277.53,0.000000,8226.10,8277.53,8256.722667,8249.97,18.066519
3,8299.20,2018-11-28,21.67,8299.20,8277.53,8226.10,8277.53,8299.20,8280.625714,8277.53,8.190490,8226.10,8299.20,8259.159333,8260.62,18.712000
4,8299.20,2018-11-29,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8283.721429,8277.53,10.573877,8236.87,8299.20,8261.596000,8271.27,19.015642
5,8299.20,2018-11-30,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8286.817143,8277.53,11.583102,8236.87,8299.20,8263.673667,8271.27,19.616521
6,8299.20,2018-12-01,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8289.912857,8299.20,11.583102,8236.87,8299.20,8265.751333,8271.27,19.977239
7,8299.20,2018-12-02,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8293.008571,8299.20,10.573877,8236.87,8299.20,8267.829000,8271.27,20.110722
8,8299.20,2018-12-03,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8296.104286,8299.20,8.190490,8236.87,8299.20,8269.906667,8271.27,20.021516
9,8314.85,2018-12-04,15.65,8299.20,8299.20,8236.87,8299.20,8299.20,8299.200000,8299.20,0.000000,8236.87,8299.20,8271.984333,8271.27,19.706597


In [10]:
# Kalendar xususiyatlari: kun/oy/chorak va dam olish flaglari (hammasi raqam)
calendar_features = features.copy()
calendar_features['day_of_week'] = calendar_features['date'].dt.dayofweek  # haftaning kuni (0=dushanba)
calendar_features['day_of_month'] = calendar_features['date'].dt.day       # oy kuni (1-31)
calendar_features['month'] = calendar_features['date'].dt.month            # oy (1-12)
calendar_features['quarter'] = calendar_features['date'].dt.quarter        # chorak (1-4)
calendar_features['is_month_start'] = calendar_features['date'].dt.is_month_start.astype(int)  # oy boshimi (1/0)
calendar_features['is_month_end'] = calendar_features['date'].dt.is_month_end.astype(int)      # oy oxirimi (1/0)
calendar_features['is_weekend'] = calendar_features['date'].dt.dayofweek.isin([5,6]).astype(int)  # dam olishmi (1/0)

# Lag/rolling + kalendar ustunlarini tozalaymiz (NaNlarni olib tashlaymiz)
calendar_features_clean = calendar_features.dropna().reset_index(drop=True)
calendar_features_clean.head()


,rate,date,diff,lag_1,lag_7,lag_30,roll_min_7,roll_max_7,roll_mean_7,roll_median_7,...,roll_mean_30,roll_median_30,roll_std_30,day_of_week,day_of_month,month,quarter,is_month_start,is_month_end,is_weekend
0,8277.53,2018-11-25,6.26,8277.53,8271.27,8226.10,8271.27,8277.53,8275.741429,8277.53,...,8253.294000,8249.97,18.682932,6,25,11,4,0,0,1
1,8277.53,2018-11-26,6.26,8277.53,8271.27,8226.10,8271.27,8277.53,8276.635714,8277.53,...,8255.008333,8249.97,18.459843,0,26,11,4,0,0,0
2,8299.20,2018-11-27,21.67,8277.53,8277.53,8226.10,8277.53,8277.53,8277.530000,8277.53,...,8256.722667,8249.97,18.066519,1,27,11,4,0,0,0
3,8299.20,2018-11-28,21.67,8299.20,8277.53,8226.10,8277.53,8299.20,8280.625714,8277.53,...,8259.159333,8260.62,18.712000,2,28,11,4,0,0,0
4,8299.20,2018-11-29,21.67,8299.20,8277.53,8236.87,8277.53,8299.20,8283.721429,8277.53,...,8261.596000,8271.27,19.015642,3,29,11,4,0,0,0


In [11]:
calendar_features_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2572 entries, 0 to 2571
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   rate            2572 non-null   float64       
 1   date            2572 non-null   datetime64[ns]
 2   diff            2572 non-null   float64       
 3   lag_1           2572 non-null   float64       
 4   lag_7           2572 non-null   float64       
 5   lag_30          2572 non-null   float64       
 6   roll_min_7      2572 non-null   float64       
 7   roll_max_7      2572 non-null   float64       
 8   roll_mean_7     2572 non-null   float64       
 9   roll_median_7   2572 non-null   float64       
 10  roll_std_7      2572 non-null   float64       
 11  roll_min_30     2572 non-null   float64       
 12  roll_max_30     2572 non-null   float64       
 13  roll_mean_30    2572 non-null   float64       
 14  roll_median_30  2572 non-null   float64       
 15  roll

In [12]:
output_path = Path('datasets/usd_rates_ready.csv')
calendar_features_clean.to_csv(output_path, index=False)
output_path

WindowsPath('datasets/usd_rates_ready.csv')